Extracting User's Google Calendar

In [19]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import datetime


In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import methods #file containing functions to interact with Google calendar like add event, create event, etc
import json
import re

google_calendar = methods.get_google_calendar_service() #get the google calendar service
tasks = methods.get_task_service() #get the task service
tasks_list = methods.extractTasks(tasks)
calendar = methods.extractCalendar(google_calendar) #extract the calendar from the service
client = methods.set_up_ChatGPT(calendar, tasks_list) #set up the ChatGPT model
now = methods.current_time()

To-Do List for the week:
1. GPT prompt engineering - make GPT refine prompt
2. Ask coe to mail degree certificate to umich
3. Jacket and bag zip repair
4. Send emails to professors for research in summer

Events for today:
- SI 649 from 4:00 PM to 5:30 PM

Logging Previous Conversation

In [22]:
def log_message(role, message):
    filename = f"conversation_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [23]:
def get_past_conversation():
    filename = f"conversation_log.txt"
    try:
        with open(filename, "r") as file:
            return file.read()
    except FileNotFoundError:
        return ""

Get User's Query

In [24]:
past_conversation = get_past_conversation()
# schedule tasks or calendar events

context = f"Here's my calendar and To-Do list for the week, store it for your reference. Calendar: {calendar}\n To-Do Tasks: {tasks_list} Today's date and time is: {now}\n"
instruction = """\nAs my virtual assistant, you are an expert with managing the calendar. You can do two things for me: schedule tasks and manage my calendar.
First, you can create, move, check, list, update, delete, and handle recurring events based on the user's requests.
You can also help me with the tasks on my to-do list by scheduling them on my calendar.
When the user wants to create an event or move an event and if the user hasn't specified when they want it scheduled, then refer to the calendar and use your best judgement to schedule the event. Keep in mind when the user would normally eat or sleep. Assume default duration for any event as 1 hour.
When the user doesn't otherwise provide enough information, you can ask for more details, upto 3 times. Otherwise use your best judgement.
Always check for any conflicts before scheduling an event.
Always ask for confirmation before making any changes to the calendar, if the user doesn't agree, then generate another suitable time slot.
When you are done, please let the user know that the task has been completed, and don't ask any more questions. If you aren't done, end with a question mark.
Second, when the user wants to schedule their to-do tasks, output all the tasks with proposed timeslots for each task. Create a balanced schedule.
Use your judgement to decide which tasks to schedule first based on the due dates and the nature of the tasks. Prioritize professional tasks over personal tasks.
Also use your best guess to determine the length of the tasks. Assume that all the tasks have to be scheduled.
If the user agrees, then add the events to the calendar and let the user know that the tasks have been scheduled.
If the user doesn't agree, then regenerate the timeslots for the tasks and ask for confirmation again.
Please perform the necessary actions using the Google Calendar API. Here's the previous conversation with the user, read it and understand the user's preferences if any, and use it to make your best judgement on scheduling.\n
Example input: 'schedule all my tasks'
Example output: 'Sure! Here's a proposed schedule for all your tasks. Is there anything you'd like to change?
- Task 1: Apr 2nd, 10:00 AM - 11:00 AM
- Task 2: Apr 2nd, 11:00 AM - 12:00 PM
- Task 3: Apr 2nd, 12:00 PM - 1:00 PM\n"""

# initial setup
initial_prompt = context + instruction + past_conversation

In [25]:
user_request = input()
combined_prompt = initial_prompt + "\nUser request: " + user_request


In [26]:
def chat_gpt(client, messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content

messages = [
    {
        "role": "system",
        "content": initial_prompt
    },
    {
        "role": "user",
        "content": user_request
    }
]

log_message("user", user_request)

In [27]:
finalized_response = False

#establishing feedback loop to chat with user
while not finalized_response:
    response = chat_gpt(client, messages)
    print(response)
    log_message("assistant", response)
    messages.append({
        "role": "assistant",
        "content": response
    })
    if re.compile(r'\?').search(response):
        finalized_response = False
        user_feedback = input()
        messages.append({
            "role": "user",
            "content": user_feedback
        })
        log_message("user", user_feedback)
    else:
        finalized_response = True


Sure! Here's a proposed schedule for all your tasks. Please let me know if you agree with the following timeslots:

1. Task: GPT prompt engineering - make GPT refine prompt
   - Proposed Time: Monday, April 22nd, 2024, from 10:30 AM to 11:30 AM

2. Task: Ask coe to mail degree certificate to UMICH
   - Proposed Time: Tuesday, April 16th, 2024, from 9:00 AM to 10:00 AM

3. Task: Jacket and bag zip repair
   - Proposed Time: Tuesday, April 16th, 2024, from 10:00 AM to 11:00 AM

4. Task: Send emails to professors for research in summer
   - Proposed Time: Tuesday, April 16th, 2024, from 11:00 AM to 12:00 PM

Do these times work for you?
Sure! Here's a revised proposed schedule for all your tasks with breaks in between and reordered by priority:

1. Task: GPT prompt engineering - make GPT refine prompt
   - Proposed Time: Monday, April 22nd, 2024, from 10:30 AM to 11:30 AM

2. Task: Send emails to professors for research in summer
   - Proposed Time: Tuesday, April 16th, 2024, from 9:00 AM

In [30]:
#get GPT to output the function calls I need to make to the calendar API
instruction2 = """\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text.
All dates should be in 'YYYY-MM-DD' format, and times should be in 24-hour format 'HH:MM'. Additionally, for functions requiring a datetime range (such as 'checkSchedule' and 'ListEvents'), the datetime should be in ISO 8601 format 'YYYY-MM-DDTHH:MM:SSZ'. Use createEvent for scheduling tasks.
Available function templates to include in the response string:
- createEvent(description, date:default=today, time, duration:default=1)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_color, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, date, time, duration, frequency, until)
- deleteRecurringEvent(event_id)
- updateRecurringEvent(event_id, new_color, new_duration, new_location)
Make sure to fill in the event IDs from the calendar provided. 
Make sure to fill in the required information, such as event names, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.
Example input: 'Schedule a team meeting for Friday afternoon at 3pm'
Example output: 'createEvent("Team Meeting", "YYYY-MM-DD", "15:00", 1)'
Example input2: 'Update my team meeting to be two hours'
Example output2: 'updateEvent("event_id", '', 2, '')
Example input3: 'Move my team meeting to tomorrow at 12pm'
Example output3: 'moveEvent("event_id", 'YYYY-MM-DD', '12:00')'
Example input4: 'Clear my afternoon'
Example output4: 'deleteEvent("event_id");deleteEvent("event_id")'
Example input5: 'Schedule all my tasks'
Example output5: 'createEvent("Task 1", "YYYY-MM-DD", "10:00", 1);createEvent("Task 2", "YYYY-MM-DD", "11:00", 1);createEvent("Task 3", "YYYY-MM-DD", "12:00", 1)\n

Now, please generate the necessary function call(s) based on the user's request."""

In [31]:
messages.append({
    "role": "system",
    "content": instruction2
})
function_call = chat_gpt(client, messages)
print(function_call)
#Example of function call: createEvent("Meeting with John", "2022-12-31", "10:00 AM", 2)

createEvent("GPT prompt engineering - make GPT refine prompt", "2024-04-22", "10:30", 1);createEvent("Send emails to professors for research in summer", "2024-04-22", "12:00", 1);createEvent("Jacket and bag zip repair", "2024-04-22", "14:00", 1);createEvent("Ask coe to mail degree certificate to UMICH", "2024-04-22", "16:00", 1)


In [12]:
function_calls = [line.strip() for line in function_call.split(';')]
print(function_calls)

# Define a dictionary mapping function names to their corresponding Python functions
functions = {
    "createEvent": methods.createEvent,
    "moveEvent": methods.moveEvent,
    "checkSchedule": methods.checkSchedule,
    "updateEvent": methods.updateEvent,
}

#TODO: works for createEvent, gotta change other functions in methods file to accept arguments as strings and convert them to appropriate data types
# Iterate through the function calls
for call in function_calls:
    # Split the string to extract function name and arguments
    if call != '':
        parts = call.split('(')
        function_name = parts[0]
        arguments = parts[1].rstrip(')').split(', ')

    # Call the corresponding function with the provided arguments
    if function_name in functions:
        functions[function_name](*arguments)
    else:
        print(f"Function '{function_name}' not found.")

['createEvent("GPT prompt engineering - make GPT refine prompt", "2024-04-22", "10:30", 1)', 'createEvent("Ask coe to mail degree certificate to umich", "2024-04-22", "09:00", 1)', 'createEvent("Jacket and bag zip repair", "2024-04-22", "11:30", 1)', 'createEvent("Send emails to professors for research in summer", "2024-04-22", "13:00", 1)']
Event created: GPT prompt engineering - make GPT refine prompt on 2024-04-22 at 10:30 for duration of 1 hours.
Event created: Ask coe to mail degree certificate to umich on 2024-04-22 at 09:00 for duration of 1 hours.
Event created: Jacket and bag zip repair on 2024-04-22 at 11:30 for duration of 1 hours.
Event created: Send emails to professors for research in summer on 2024-04-22 at 13:00 for duration of 1 hours.
